In [1]:
import tensorflow as tf
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import datetime
import time

In [ ]:
"""
read data (processed_data)
"""
idxname = pd.read_csv("data/index.csv", sep='\t')
stkname = pd.read_csv("data/stocks.csv", sep='\t')

indicators = []
stocks = []
for filename in idxname.filename:
    indicators.append(pd.read_csv("processed_data/index/"+filename))
for filename in stkname.symbol:
    stocks.append(pd.read_csv("processed_data/stocks/"+filename))

#print(indicators[0])
#print(stocks[0])

"""
Data preparation
"""
max0day = int(stocks[0].loc[stocks[0].iloc[:,1]==0].Date.max())

#stocks[0].iloc[max0day:,1:5].plot()
#plt.show()
data = stocks[0].iloc[max0day+1:].Close

train = data.iloc[:-730]
test = data.iloc[-730:]

In [2]:
"""
read data (original)
"""
idxname = pd.read_csv("data/index.csv", sep='\t')
stkname = pd.read_csv("data/stocks.csv", sep='\t')

indicators = []
stocks = []
for filename in idxname.filename:
    indicators.append(pd.read_csv("data/index/"+filename).fillna(0))
for filename in stkname.symbol:
    stocks.append(pd.read_csv("data/stocks/"+filename).fillna(0))

#print(indicators[0])
#print(stocks[0])

"""
Data preparation
"""
data = stocks[0].Close

train = data.iloc[:-730]
test = data.iloc[-730:]

In [3]:
train = train.values
test = test.values

In [4]:
train_x = np.array([train[i:i+365] for i in range(len(train)-365)])[:,:, np.newaxis]
train_y = np.array([train[i+365] for i in range(len(train)-365)])[:, np.newaxis]

test_x = np.array([test[i:i+365] for i in range(len(test)-365)])[:,:, np.newaxis]
test_y = np.array([test[i+365] for i in range(len(test)-365)])[:, np.newaxis]

# print(train_x)
# print(len(train_x))
# print(train_y)
# print(len(train_y))
print(train_x.shape)
print(train_y.shape)

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
data_gen = TimeseriesGenerator(train, train, length=365, batch_size=128)

#x, y = data_gen[0]
#print(x)
#print(len(x))
#print(y)
#print(len(y))

(8648, 365, 1)
(8648, 1)


In [5]:
"""
LSTM
"""
model = tf.keras.Sequential()
#model.add(layers.BatchNormalization())
# (i)
model.add(layers.LSTM(128, return_sequences=True, dropout=.5,
                     input_shape=(None, 1)))
#model.add(layers.BatchNormalization())
model.add(layers.LSTM(64))
#model.add(layers.Flatten())
model.add(layers.Dense(1))
#model.add(layers.CuDNNLSTM(128))
# (ii)
#model.add(layers.Bidirectional(layers.LSTM(128), return_sequences=True))
#model.add(layers.Bidirectional(layers.LSTM(64)))

adam = tf.keras.optimizers.Adam(lr=0.001)
model.compile(loss='mean_squared_error', optimizer=adam, metrics=['mse'])
#model.compile(loss='mse', optimizer=adam, metrics=['mse'])

model.build()
print(model.summary())

model.fit(train_x, train_y,
          shuffle = True,
          epochs=200)

# model.fit_generator(data_gen, 
#                     epochs=200, 
#                     shuffle=True)

W0806 15:45:38.684093 140135000602432 deprecation.py:506] From /home/c405/venv/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0806 15:45:39.046806 140135000602432 deprecation.py:323] From /home/c405/venv/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 128)         66560     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 116,033
Trainable params: 116,033
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 1139.5753 - mean_squared_error: 1139.5752
Epoch 2/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 925.0460 - mean_squared_error: 925.0460
Epoch 3/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 813.7425 - mean_squared_error: 813.7425
Epoch 4/200
8

8648/8648 [==============================] - 33s 4ms/sample - loss: 184.8670 - mean_squared_error: 184.8670
Epoch 63/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 146.3998 - mean_squared_error: 146.3998
Epoch 64/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 135.5170 - mean_squared_error: 135.5170
Epoch 65/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 140.8291 - mean_squared_error: 140.8291
Epoch 66/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 127.9640 - mean_squared_error: 127.9640
Epoch 67/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 150.1271 - mean_squared_error: 150.1271
Epoch 68/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 112.4483 - mean_squared_error: 112.4483
Epoch 69/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 103.8458 - mean_squared_error: 103.8458
Epoch 70/200
8648/8648 [=====================

8648/8648 [==============================] - 33s 4ms/sample - loss: 72.4774 - mean_squared_error: 72.4774
Epoch 131/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 79.9298 - mean_squared_error: 79.9298
Epoch 132/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 77.8554 - mean_squared_error: 77.8554
Epoch 133/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 81.1149 - mean_squared_error: 81.1149
Epoch 134/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 83.0514 - mean_squared_error: 83.0514
Epoch 135/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 74.6913 - mean_squared_error: 74.6913
Epoch 136/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 76.7583 - mean_squared_error: 76.7583
Epoch 137/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 73.3050 - mean_squared_error: 73.3050
Epoch 138/200
8648/8648 [=============================

8648/8648 [==============================] - 33s 4ms/sample - loss: 69.8642 - mean_squared_error: 69.8642
Epoch 199/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 57.5954 - mean_squared_error: 57.5954
Epoch 200/200
8648/8648 [==============================] - 33s 4ms/sample - loss: 72.2892 - mean_squared_error: 72.2892


In [7]:
score = model.evaluate(test_x, test_y)
print(score)
pred_y = model.predict(test_x)
from sklearn.metrics import mean_squared_error
from math import sqrt
print(sqrt(mean_squared_error(test_y, pred_y)))

365/365 [==============================] - 2s 5ms/sample - loss: 9112.2222 - mean_squared_error: 9112.2217
[9112.222177333048, 9112.222]
95.45796099376123


In [ ]:
def smape_kun(y_true, y_pred):
    return np.mean((np.abs(np.array(y_pred) - np.array(y_true)) * 200/ (np.abs(y_pred) + np.abs(y_true))))
error2 = smape_kun(test_ar, predictions)
print('Symmetric mean absolute percentage error: %.3f' % error2)

In [ ]:
#plt.figure(figsize=(12,7))
%matplotlib qt

#plt.plot(data, 'green', color='blue', label='Training Data')
plt.plot(test.index, predictions, 
         color='green', 
         marker=',', #'o', 
         linestyle='dashed', 
         label='Predicted Price')
plt.plot(test.index, test, color='red', label='Actual Price')
plt.title('Prices Prediction')
plt.xlabel('Dates')
plt.ylabel('Prices')
#plt.xticks(np.arange(0,1857, 300), stocks[0]['Date'][0:1857:300])
#plt.xticks(np.arange(max0day+1,len(data), 300), stocks[0]['Date'][max0day+1:len(data):300])
plt.legend()

The graph shows that the trained model just picks the same price of today as the price of the next day.